In [1]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from eli5.sklearn import PermutationImportance
from catboost import CatBoostRegressor,Pool
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import norm
from scipy import linalg
from sklearn import tree
from sklearn import svm
import lightgbm as lgb
#import xgboost as xgb
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import graphviz
import warnings
import random
import eli5
import shap  # package used to calculate Shap values
import time
import glob
import sys
import os
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression

/Users/wangrunqiu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/wangrunqiu/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
warnings.filterwarnings("ignore")

print("Reading csv files...")
train = pd.read_csv('./train.csv' , dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})
print("Train: rows:{} columns:{}".format(train.shape[0], train.shape[1]))

Reading csv files...
Train: rows:629145480 columns:2


In [4]:
rows = 150_000
segments = int(np.floor(train.shape[0] / rows)) # 4194

X_train = pd.DataFrame(index=range(segments), dtype=np.float64) # 使用对应的特征
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])
submission = pd.read_csv('./sample_submission.csv', index_col='seg_id')

X_train =pd.read_csv('./data/4194samples/X_train_138features_filtered.csv',index_col=0)
X_test = pd.read_csv('./data/4194samples/X_test_138features_filtered.csv',index_col=0)

#y_train

scale = 1  # test:使滑动窗口缩小10倍
for segment in range(segments):
    # y_train and X_train
    #     if segment * rows / scale + rows > train.shape[0]:
    #         break
    seg = train.iloc[int(segment * rows / scale): int(segment * rows / scale + rows)]
    x = pd.Series(seg['acoustic_data'].values)
    y = seg['time_to_failure'].values[-1]
    y_train.loc[segment, 'time_to_failure'] = y
#print(X_train.head())
#print(X_test.head())
print(y_train.shape)
#print("submission has: rows:{} cols:{}".format(submission.shape[0], submission.shape[1]))
print("Reading ends.")

(4194, 1)
Reading ends.


In [5]:
#VIF
print('start')
from statsmodels.stats.outliers_influence import variance_inflation_factor
def filter_column_based_on_VIF(X):
    '''X - pandas dataframe'''
    thresh = 10.0
    variables = list(range(X.shape[1]))
        
    vif = [variance_inflation_factor(X.values, ix) for ix in range(X.shape[1])]
    cols = []
    for col, cont in enumerate(vif):
        if cont < thresh:
            cols.append(X.columns.values[col])
    
    print('Remaining variables: %s'%cols)
    return cols
remaining_cols = filter_column_based_on_VIF(X_train)
X_train_new = X_train[remaining_cols]
X_test_new = X_test[remaining_cols]

print(X_train_new.shape)
print(X_test_new.shape)
print('end')

start
Remaining variables: ['skew', 'mean_change_abs', 'mean_change_rate_first_10000', 'mean_change_rate_last_10000', 'av_change_abs_roll_std_10', 'av_change_abs_roll_mean_10', 'av_change_abs_roll_std_100', 'av_change_abs_roll_mean_100', 'av_change_abs_roll_std_1000']
(4194, 9)
(2624, 9)
end


In [6]:
# 正则化
print("FillNA and standardScalar...")
means_dict = {}
for col in X_train_new.columns:
    if X_train_new[col].isnull().any():
        print(col)
        mean_value = X_train_new.loc[X_train_new[col] != -np.inf, col].mean()
        X_train_new.loc[X_train_new[col] == -np.inf, col] = mean_value
        X_train_new[col] = X_train_new[col].fillna(mean_value)
        means_dict[col] = mean_value

for col in X_test_new.columns:
    if X_test_new[col].isnull().any():
        X_test_new.loc[X_test_new[col] == -np.inf, col] = means_dict[col]
        X_test_new[col] = X_test_new[col].fillna(means_dict[col])

X = X_train_new.copy()
y = y_train.copy()
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
scaler = StandardScaler()
scaler.fit(X_train_new)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_new), columns=X_train_new.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_new), columns=X_test_new.columns)
print("FillNA and standardScalar ends.")

FillNA and standardScalar...
FillNA and standardScalar ends.


In [7]:
#cross validation
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)
X=X_train_scaled
X_test=X_test_scaled
y=y_train
oof = np.zeros(len(X))
prediction = np.zeros(len(X_test))
scores = []
feature_importance = pd.DataFrame()

In [8]:
print('start')
from sklearn.model_selection import GridSearchCV
param_test1 = {'n_estimators':range(1,9,1)}
gsearch1 = GridSearchCV(estimator = RandomForestRegressor(), param_grid = param_test1, scoring=None,cv=5)
gsearch1.fit(X,y)
GridSearchCV(cv=5, error_score='raise',estimator=RandomForestRegressor(bootstrap=True,criterion='mae',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': range(10, 71, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='none', verbose=0)

print(gsearch1.best_params_)
print(gsearch1.best_score_)
print('end')



start
{'n_estimators': 8}
0.0665546518866079
end
start
{'max_depth': 4, 'min_samples_leaf': 46, 'min_samples_split': 9}
0.21490697655000435
end
start
2.5405896252233964
end
